# Import libraries and functions

In [1]:
import math
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

## Initialize Orekit and import Orekit libraries

In [2]:
import orekit
vm = orekit.initVM()
print ('Java version:',vm.java_version)
print ('Orekit version:', orekit.VERSION)

Java version: 1.8.0_152-release
Orekit version: 12.0.1


In [3]:
from orekit.pyhelpers import setup_orekit_curdir, download_orekit_data_curdir
setup_orekit_curdir('../../orekit-data.zip')

In [4]:
from java.util import Arrays
from orekit import JArray_double

In [5]:
from org.orekit.propagation.analytical.tle import TLE, TLEPropagator
from org.orekit.frames import FramesFactory, LOFType, LocalOrbitalFrame
from org.orekit.time import AbsoluteDate, TimeScalesFactory
from org.orekit.utils import Constants

# Load datasets

## Space-Track dataset

In [19]:
reduced, frac1 = False, 0.25
reduced_sample_alt_e, frac2, min_alt, max_alt, e_thres, sampled1 = False, 1.0, 500, 600, 0.2, False 
reduced_sample_leos, frac3, leo, sampled2,  = True, 0.25, 'leo4', True  # smallest LEO
if reduced:
    filepath = f"../datasets/space-track-dataset-reduced-{int(frac1*100)}.csv"
elif reduced_sample_alt_e:
    if sampled1:
        filepath = f"../datasets/space-track-dataset-reduced-{int(frac2*100)}-h-{min_alt}-{max_alt}-e-{int(e_thres*100)}.csv"
    else:
        filepath = f"../datasets/space-track-dataset-reduced-h-{min_alt}-{max_alt}-e-{int(e_thres*100)}.csv"
elif reduced_sample_leos:
    if sampled2:
        filepath = f"../datasets/space-track-dataset-{leo}-reduced-{int(frac3*100)}.csv"
    else:
        filepath = f"../datasets/space-track-dataset-{leo}.csv"
else:
     filepath = '../datasets/space-track-dataset.csv'

st_df = pd.read_csv(filepath,nrows=101, memory_map=True)
st_df.head()

,NORAD_CAT_ID,OBJECT_NAME,OBJECT_ID,DECAY_DATE,EPOCH_DATE,EPOCH_TIME,CREATION_DATE,CENTER_NAME,REF_FRAME,TIME_SYSTEM,...,MEAN_MOTION_DOT,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,TLE_LINE1,TLE_LINE2
0,1585,COSMOS 87,1965-073B,NaN,2023-12-28,00:01:58.833696,2023-12-28T18:10:27,EARTH,TEME,UTC,...,-9.100000e-07,0.0,7853.347,115.436,1633.103,1317.320,PAYLOAD,MEDIUM,1 01585U 65073B 23362.00137539 -.00000091 0...,2 01585 56.0591 188.4948 0201050 256.1861 273...
1,14879,THORAD DELTA 1 DEB,1974-089ES,NaN,2023-12-28,00:02:43.493856,2023-12-28T18:10:27,EARTH,TEME,UTC,...,5.300000e-07,0.0,7990.496,118.473,1783.498,1441.225,DEBRIS,SMALL,1 14879U 74089ES 23362.00189229 .00000053 0...,2 14879 101.2066 12.9018 0214175 298.1690 85...
2,22999,COSMOS 2268,1994-011A,NaN,2023-12-28,00:45:36.477216,2023-12-28T06:46:17,EARTH,TEME,UTC,...,3.400000e-07,0.0,7796.173,114.178,1426.505,1409.571,PAYLOAD,LARGE,1 22999U 94011A 23362.03167219 .00000034 0...,2 22999 82.5759 195.1130 0010860 82.8280 85...
3,12854,SL-12 DEB,1970-103G,NaN,2023-12-28,00:59:43.669824,2023-12-28T18:10:27,EARTH,TEME,UTC,...,4.929500e-04,0.0,7833.836,115.006,1699.140,1212.262,DEBRIS,MEDIUM,1 12854U 70103G 23362.04147766 .00049295 0...,2 12854 50.2636 298.2887 0310753 359.8240 0...
4,45162,ONEWEB-0059,2020-008AH,NaN,2023-12-28,01:01:27.212448,2023-12-28T06:26:17,EARTH,TEME,UTC,...,5.620000e-06,0.0,7599.848,109.892,1222.821,1220.606,PAYLOAD,LARGE,1 45162U 20008AH 23362.04267607 .00000562 0...,2 45162 87.8906 215.4662 0001457 92.6847 267...


## ESA DISCOS satellite dataset

In [20]:
esa_df = pd.read_csv('../datasets/esa-discos-satellite-data.csv',usecols=['satno', 'constellationDiscosID'], memory_map=True)
print(f"Number of constellations: {esa_df['constellationDiscosID'].unique().shape[0]}")
esa_df.head()

Number of constellations: 28


,satno,constellationDiscosID
0,739,NaN
1,1585,NaN
2,1729,NaN
3,7148,NaN
4,8835,NaN


## Extend Space-Track dataset with DISCOS constellation ID for each satellite in Space-Track dataset

In [21]:
df = pd.merge(st_df, esa_df, how='left', left_on='NORAD_CAT_ID', right_on='satno').drop('satno', axis=1).rename(columns={'constellationDiscosID':'CONSTELLATION_DISCOS_ID'})
df.columns

Index(['NORAD_CAT_ID', 'OBJECT_NAME', 'OBJECT_ID', 'DECAY_DATE', 'EPOCH_DATE',
       'EPOCH_TIME', 'CREATION_DATE', 'CENTER_NAME', 'REF_FRAME',
       'TIME_SYSTEM', 'MEAN_ELEMENT_THEORY', 'MEAN_MOTION', 'ECCENTRICITY',
       'INCLINATION', 'RA_OF_ASC_NODE', 'ARG_OF_PERICENTER', 'MEAN_ANOMALY',
       'EPHEMERIS_TYPE', 'CLASSIFICATION_TYPE', 'REV_AT_EPOCH', 'BSTAR',
       'MEAN_MOTION_DOT', 'MEAN_MOTION_DDOT', 'SEMIMAJOR_AXIS', 'PERIOD',
       'APOAPSIS', 'PERIAPSIS', 'OBJECT_TYPE', 'RCS_SIZE', 'TLE_LINE1',
       'TLE_LINE2', 'CONSTELLATION_DISCOS_ID'],
      dtype='object')

In [22]:
print(f"Number of constellations: {df['CONSTELLATION_DISCOS_ID'].unique().shape[0]}")
df.head()

Number of constellations: 5


,NORAD_CAT_ID,OBJECT_NAME,OBJECT_ID,DECAY_DATE,EPOCH_DATE,EPOCH_TIME,CREATION_DATE,CENTER_NAME,REF_FRAME,TIME_SYSTEM,...,MEAN_MOTION_DDOT,SEMIMAJOR_AXIS,PERIOD,APOAPSIS,PERIAPSIS,OBJECT_TYPE,RCS_SIZE,TLE_LINE1,TLE_LINE2,CONSTELLATION_DISCOS_ID
0,1585,COSMOS 87,1965-073B,NaN,2023-12-28,00:01:58.833696,2023-12-28T18:10:27,EARTH,TEME,UTC,...,0.0,7853.347,115.436,1633.103,1317.320,PAYLOAD,MEDIUM,1 01585U 65073B 23362.00137539 -.00000091 0...,2 01585 56.0591 188.4948 0201050 256.1861 273...,NaN
1,14879,THORAD DELTA 1 DEB,1974-089ES,NaN,2023-12-28,00:02:43.493856,2023-12-28T18:10:27,EARTH,TEME,UTC,...,0.0,7990.496,118.473,1783.498,1441.225,DEBRIS,SMALL,1 14879U 74089ES 23362.00189229 .00000053 0...,2 14879 101.2066 12.9018 0214175 298.1690 85...,NaN
2,22999,COSMOS 2268,1994-011A,NaN,2023-12-28,00:45:36.477216,2023-12-28T06:46:17,EARTH,TEME,UTC,...,0.0,7796.173,114.178,1426.505,1409.571,PAYLOAD,LARGE,1 22999U 94011A 23362.03167219 .00000034 0...,2 22999 82.5759 195.1130 0010860 82.8280 85...,NaN
3,12854,SL-12 DEB,1970-103G,NaN,2023-12-28,00:59:43.669824,2023-12-28T18:10:27,EARTH,TEME,UTC,...,0.0,7833.836,115.006,1699.140,1212.262,DEBRIS,MEDIUM,1 12854U 70103G 23362.04147766 .00049295 0...,2 12854 50.2636 298.2887 0310753 359.8240 0...,NaN
4,45162,ONEWEB-0059,2020-008AH,NaN,2023-12-28,01:01:27.212448,2023-12-28T06:26:17,EARTH,TEME,UTC,...,0.0,7599.848,109.892,1222.821,1220.606,PAYLOAD,LARGE,1 45162U 20008AH 23362.04267607 .00000562 0...,2 45162 87.8906 215.4662 0001457 92.6847 267...,7.0


## Remove fractional part of seconds in EPOCH_TIME

In [23]:
df['EPOCH_TIME'] = df['EPOCH_TIME'].apply(lambda t: t.split('.')[0])
df['EPOCH_TIME'].head()

0    00:01:58
1    00:02:43
2    00:45:36
3    00:59:43
4    01:01:27
Name: EPOCH_TIME, dtype: object

# Build Space-Track dynamic satellite graph from 2023-12-28 to 2024-01-28

In [24]:
def generate_timestamps(start_date, end_date, step_time_sec):
    duration = (end_date-start_date).total_seconds()  # in seconds

    timestamps = [start_date + timedelta(seconds=dt) for dt in np.arange(0, duration, step_time_sec)]
    return timestamps

In [25]:
def datetime_from(absolute_date):
    dt = absolute_date.getComponents(0)
    d = dt.getDate()
    t = dt.getTime()
    return datetime(d.getYear(), d.getMonth(), d.getDay(), t.getHour(), t.getMinute(), int(t.getSecond()))

In [26]:
leo1_limits = (0.4, 44, 51) 
leo2_limits = (0.4, 25, 25)
leo3_limits = (0.4, 12, 12)
leo4_limits = (0.4, 2, 2)

In [27]:
def gen_weights(size):
    weights = np.random.rand(size)
    weights[size-1] += 0.5 
    weights.sort()
    weights /= weights.sum()
    
    return weights
w = gen_weights(3)
# radial least covariance == more certainty == heavier weight
# in-track more covariance == least certainty == lighter weight
limit_weights = (w[2], w[0], w[1]) 
limit_weights

(0.5196239466228537, 0.1544883527301125, 0.3258877006470339)

In [28]:
def date_time(epoch_date, epoch_time):
    d = epoch_date.split('-')
    year, month, day = int(d[0]), int(d[1]), int(d[2])

    t = epoch_time.split(':')
    hour, minute, second = int(t[0]), int(t[1]), int(t[2])
    return datetime(year, month, day, hour, minute, second)

def combine_date_time(row):
    return date_time(row[4], row[5])

def duration_from(date, target):
    return (date-target).total_seconds()

def absolute_date_from(date_time):
    return AbsoluteDate(date_time.year, date_time.month, date_time.day, date_time.hour, date_time.minute, float(date_time.second), TimeScalesFactory.getUTC())

In [29]:
def rev_day_from(rad_per_sec):
    revolutions_per_day = (rad_per_sec * 24 * 3600) / (2 * math.pi)
    return revolutions_per_day

def rev_day2_from(rad_per_sec2):
    revolutions_per_day2 = (rad_per_sec2 * (24 * 3600) ** 2) / ((2 * math.pi) ** 2 * 3600)
    return revolutions_per_day2

def degrees_from(radians):
    degrees = radians * (180 / math.pi)
    return degrees

def m_from(km):
    return km * 1000

def km_from(m):
    return m / 1000

def min_from(seconds):
    return seconds / 60

In [30]:
def propagate_tle(tle_line1, tle_line2, date):
    tle = TLE(tle_line1, tle_line2)
    propagator = TLEPropagator.selectExtrapolator(tle)
    
    state = propagator.propagate(absolute_date_from(date))
    
    initial_frame = state.getFrame()
    lof = LocalOrbitalFrame(initial_frame, LOFType.QSW, propagator, str(tle.getSatelliteNumber())+"_lof")
    transformer = initial_frame.getTransformTo(lof, tle.getDate())
    
    propagated_tle = TLE.stateToTLE(state, tle,  propagator.getDefaultTleGenerationAlgorithm(tle.getUtc(), state.getFrame()))
    
    return propagated_tle, state, transformer

def update_nodes_data(nodes_data, data):
    for k in nodes_data:
        nodes_data[k].append(data[k])

def timestamp_data(tle_df, ids, timestamp, nodes_data, nodes_data_keys, earth_rad_km):
    t_data = []
    for sat_id in ids:
        sat_df = tle_df[tle_df['NORAD_CAT_ID'] == sat_id]
        
        sat_ser = sat_df.iloc[0]
        cat_cols = ('NORAD_CAT_ID', 'OBJECT_NAME', 'OBJECT_ID', 'DECAY_DATE', 'CENTER_NAME', 'REF_FRAME', 'TIME_SYSTEM', 'MEAN_ELEMENT_THEORY', 'EPHEMERIS_TYPE', 'CLASSIFICATION_TYPE', 'OBJECT_TYPE', 'RCS_SIZE', 'CONSTELLATION_DISCOS_ID')
        node_data = {col:sat_ser[col] for col in cat_cols}
        
        sat_data = (sat_df[['CONSTELLATION_DISCOS_ID', 'NORAD_CAT_ID', 'TLE_LINE1', 'TLE_LINE2', 'EPOCH_DATE', 'EPOCH_TIME']]).to_numpy()
        
        date_time_date_col = np.apply_along_axis(combine_date_time, 1, sat_data)
        sat_data = np.column_stack((sat_data[:, 0:4], date_time_date_col))
        closest_idx = np.argmin(np.abs(np.vectorize(duration_from)(sat_data[:, 4], timestamp)))
        
        tle, state, transformer = propagate_tle(sat_data[closest_idx, 2], sat_data[closest_idx, 3], timestamp)
        a = km_from(state.getA()) # in km to keep consistent with space-track
        e = tle.getE()
        pv = state.getPVCoordinates()
        pos, vel = pv.getPosition(), pv.getVelocity()
        
        # orekit units converted to space-track units
        tle_data = [rev_day_from(tle.getMeanMotion()), e, degrees_from(tle.getI()), degrees_from(tle.getRaan()), degrees_from(tle.getPerigeeArgument()), degrees_from(tle.getMeanAnomaly()), tle.getRevolutionNumberAtEpoch(), tle.getBStar(), rev_day2_from(tle.getMeanMotionFirstDerivative()), rev_day2_from(tle.getMeanMotionSecondDerivative()), a, min_from(state.getKeplerianPeriod()), a*(1+e)-earth_rad_km, a*(1-e)-earth_rad_km]
        pv_data = [km_from(pos.x), km_from(pos.y), km_from(pos.z), km_from(vel.x), km_from(vel.y), km_from(vel.z), timestamp]
        tle_data.extend(pv_data)
        
        num_cols = [col for col in nodes_data_keys if col not in cat_cols]
        node_data.update({col:val for col, val in zip(num_cols, tle_data)})
        
        update_nodes_data(nodes_data, node_data)
    
        t_data.append((sat_data[0, 0], sat_id, pv, transformer))
    return t_data

def conjunction(pv1, pv2, limits, weighted=False):
    distance = -1
    (r_lim, r_weight), (it_lim, it_weight), (ct_lim, ct_weight) = limits
    pos1 = pv1.getPosition()
    pos2 = pv2.getPosition()
    radial1, in_track1, cross_track1 = pos1.x, pos1.y, pos1.z
    radial2, in_track2, cross_track2 = pos2.x, pos2.y, pos2.z
    r_dist = math.fabs(radial1 - radial2)
    it_dist = math.fabs(in_track1 - in_track2)
    ct_dist = math.fabs(cross_track1 - cross_track2)

    if weighted:
        # WEIGHTED CONDITION
        if r_weight*(r_dist <= r_lim) + it_weight*(it_dist <= it_lim) + ct_weight*(ct_dist <= ct_lim) > 0.5:
            distance = r_weight*(r_dist ** 2) + r_weight*(it_dist ** 2) + r_weight*(ct_dist ** 2)
    else:
        if r_dist <= r_lim or it_dist <= it_lim or ct_dist <= ct_lim:
            distance = (r_dist ** 2) + (it_dist ** 2) + (ct_dist ** 2)

    return r_dist, it_dist, ct_dist, math.sqrt(distance) if distance >= 0 else distance

def update_edges(edges, data):
    for k in edges:
        edges[k].append(data[k])

def check_conjunction(sat1_id, pv1_lof, sat2_id, pv2_lof, limits, date_time, prop, edges):
    updated = False
    r_dist, it_dist, ct_dist, distance = conjunction(pv1_lof, pv2_lof, limits)
    if distance >= 0:
        data = {'source': sat1_id, 'target': sat2_id, 'weight': 1, 'r_dist': r_dist, 'it_dist': it_dist,
                       'ct_dist': ct_dist, 'dist': distance, 'timestamp':date_time, 'prop': prop}
        update_edges(edges, data)
        updated = True
    return updated

def tle_to_edges(tle_df, tle_df_cols, timestamps, ids, limits, earth_rad_km):
    edges = {'source':[], 'target':[], 'weight':[], 'r_dist':[], 'it_dist':[], 'ct_dist':[], 'dist':[], 'timestamp':[], 'prop':[]}
    nodes_data = {col:[] for col in tle_df_cols if col not in ('EPOCH_DATE', 'EPOCH_TIME', 'CREATION_DATE', 'TLE_LINE1', 'TLE_LINE2')}
    nodes_data.update({col:[] for col in ['PX', 'PY', 'PZ', 'VX', 'VY', 'VZ', 'TIMESTAMP']})
    
    num_satellites = ids.shape[0]
    for timestamp in tqdm(timestamps):
        t_data = timestamp_data(tle_df, ids, timestamp, nodes_data, list(nodes_data.keys()), earth_rad_km)
            
        updated = False
        for i in range(num_satellites):
            constellation1_id, sat1_id, pv1, transformer = t_data[i]
            pv1_lof = transformer.transformPVCoordinates(pv1)
            for j in range(i+1, num_satellites):
                constellation2_id, sat2_id, pv2, _ = t_data[j]
                if constellation1_id == constellation2_id:
                    continue
                
                pv2_lof = transformer.transformPVCoordinates(pv2)
                updated |= check_conjunction(sat1_id, pv1_lof, sat2_id, pv2_lof, limits, timestamp, True, edges)
        if not updated:
            data = {'source': None, 'target': None, 'weight': None, 'r_dist': None, 'it_dist': None,
                       'ct_dist': None, 'dist': None, 'timestamp':timestamp, 'prop': None}
            update_edges(edges, data)
        
    return pd.DataFrame(edges), pd.DataFrame(nodes_data)

In [31]:
start_date = datetime(2023, 12, 28, 0, 0, 0)
end_date = datetime(2024, 1, 28, 0, 0, 0)
step_time_sec = 60 * 60  # hour by hour in seconds

timestamps = generate_timestamps(start_date, end_date, step_time_sec)
len(timestamps)

limits = tuple(map(m_from, leo4_limits))
sat_ids = df['NORAD_CAT_ID'].unique()
sat_ids.sort()
edges_df, node_feats_df = tle_to_edges(df, list(df.columns), timestamps, sat_ids, tuple(zip(limits, limit_weights)), km_from(Constants.WGS84_EARTH_EQUATORIAL_RADIUS))
edges_df.head()

  0%|          | 0/744 [00:00<?, ?it/s]

,source,target,weight,r_dist,it_dist,ct_dist,dist,timestamp,prop
0,2223.0,19909.0,1.0,3.231525e+01,2.494367e+05,2.052567e+06,2.067667e+06,2023-12-28,True
1,5552.0,16455.0,1.0,4.859706e+06,3.241714e+06,2.502273e+02,5.841699e+06,2023-12-28,True
2,6125.0,11136.0,1.0,5.061006e+06,1.582337e+03,2.342877e+06,5.576993e+06,2023-12-28,True
3,6680.0,56076.0,1.0,8.145522e+06,1.216780e+07,1.920507e+03,1.464257e+07,2023-12-28,True
4,11050.0,23412.0,1.0,8.104240e+04,1.660923e+06,1.166244e+03,1.662900e+06,2023-12-28,True


In [32]:
src_df = pd.merge(edges_df['source'], esa_df, how='left', left_on='source', right_on='satno').drop('satno', axis=1).rename(columns={'constellationDiscosID':'src_constellationDiscosID'})
tgt_df = pd.merge(edges_df['target'], esa_df, how='left', left_on='target', right_on='satno').drop('satno', axis=1).rename(columns={'constellationDiscosID':'tgt_constellationDiscosID'})
src_tgt_df = pd.concat([src_df, tgt_df], axis=1)
src_tgt_df

,source,src_constellationDiscosID,target,tgt_constellationDiscosID
0,2223.0,NaN,19909.0,NaN
1,5552.0,NaN,16455.0,NaN
2,6125.0,NaN,11136.0,NaN
3,6680.0,NaN,56076.0,7.0
4,11050.0,NaN,23412.0,NaN
...,...,...,...,...
2344,13767.0,NaN,48795.0,7.0
2345,37190.0,4.0,45154.0,7.0
2346,6853.0,NaN,15471.0,NaN
2347,7679.0,NaN,11047.0,NaN


In [33]:
node_feats_df.head()

,NORAD_CAT_ID,OBJECT_NAME,OBJECT_ID,DECAY_DATE,CENTER_NAME,REF_FRAME,TIME_SYSTEM,MEAN_ELEMENT_THEORY,MEAN_MOTION,ECCENTRICITY,...,OBJECT_TYPE,RCS_SIZE,CONSTELLATION_DISCOS_ID,PX,PY,PZ,VX,VY,VZ,TIMESTAMP
0,1585,COSMOS 87,1965-073B,NaN,EARTH,TEME,UTC,SGP4,12.474427,0.020105,...,PAYLOAD,MEDIUM,NaN,7569.427368,-303.619554,2107.196293,1.587111,4.062479,-5.624482,2023-12-28
1,2223,DELTA 1 DEB,1966-016C,NaN,EARTH,TEME,UTC,SGP4,12.877150,0.009348,...,DEBRIS,SMALL,NaN,4443.610779,2601.235853,5666.806467,-3.377597,-4.322483,4.707343,2023-12-28
2,4499,DELTA 1 DEB,1968-069G,NaN,EARTH,TEME,UTC,SGP4,12.512624,0.002789,...,DEBRIS,SMALL,NaN,-6143.585283,-4849.187522,682.242265,-1.348347,0.739122,-6.948606,2023-12-28
3,5216,COSMOS 417,1971-041G,NaN,EARTH,TEME,UTC,SGP4,12.614277,0.009526,...,PAYLOAD,MEDIUM,NaN,1886.155793,7084.301025,-2543.273017,-2.587492,-1.581241,-6.513951,2023-12-28
4,5552,COSMOS 449,1971-086F,NaN,EARTH,TEME,UTC,SGP4,12.387134,0.003800,...,PAYLOAD,MEDIUM,NaN,-701.720748,7232.369282,-3015.365872,-1.860979,-2.806877,-6.284337,2023-12-28


## Timestamps without edges

In [32]:
(edges_df[edges_df['source'].isnull()]['timestamp']).unique()

<DatetimeArray>
['2023-12-30 19:00:00']
Length: 1, dtype: datetime64[ns]

## Summary of satellite graph

In [34]:
datetime_grouped_edges_df = edges_df.groupby('timestamp')
summary = datetime_grouped_edges_df.agg(
    num_edges=('source', 'size'),
    num_unique_src=('source', 'nunique'),
    num_unique_tgt=('target', 'nunique')
)
summary['num_nodes'] = datetime_grouped_edges_df.apply(lambda x: len(set(x['source']) | set(x['target'])), include_groups=False)
summary['graph_density'] = datetime_grouped_edges_df.apply(lambda x: len(x) / (len(set(x['source']) | set(x['target']))*(len(set(x['source']) | set(x['target']))-1)),include_groups=False)
summary

,num_edges,num_unique_src,num_unique_tgt,num_nodes,graph_density
timestamp,,,,,
2023-12-28 00:00:00,6,6,6,11,0.054545
2023-12-28 01:00:00,6,5,6,11,0.054545
2023-12-28 02:00:00,1,1,1,2,0.500000
2023-12-28 03:00:00,5,5,5,10,0.055556
2023-12-28 04:00:00,3,3,2,5,0.150000
...,...,...,...,...,...
2024-01-27 19:00:00,2,2,2,4,0.166667
2024-01-27 20:00:00,3,3,3,6,0.100000
2024-01-27 21:00:00,1,1,1,2,0.500000


In [35]:
summary.describe()

,num_edges,num_unique_src,num_unique_tgt,num_nodes,graph_density
count,744.000000,744.000000,744.000000,744.000000,744.000000
mean,3.157258,3.021505,3.047043,6.104839,0.177000
std,1.686220,1.700153,1.699241,3.154143,0.158862
min,1.000000,0.000000,0.000000,2.000000,0.029240
25%,2.000000,2.000000,2.000000,4.000000,0.071429
50%,3.000000,3.000000,3.000000,6.000000,0.100000
75%,4.000000,4.000000,4.000000,8.000000,0.166667
max,10.000000,10.000000,10.000000,19.000000,0.500000


## Summary of satellite graph node features

In [40]:
if reduced:
    edges_savepath = f"../datasets/space-track-ap2-graph-edges-reduced-{int(frac1 * 100)}.csv"
elif reduced_sample_alt_e:
    if sampled1:
        edges_savepath = f"../datasets/space-track-ap2-graph-edges-reduced-{int(frac2 * 100)}-h-{min_alt}-{max_alt}-e-{int(e_thres * 100)}.csv"
    else:
        edges_savepath = f"../datasets/space-track-ap2-graph-edges-reduced-h-{min_alt}-{max_alt}-e-{int(e_thres * 100)}.csv"
elif reduced_sample_leos:
    if sampled2:
        edges_savepath = f"../datasets/space-track-ap2-graph-edges-{leo}-reduced-{int(frac3 * 100)}.csv"
    else:
        edges_savepath = f"../datasets/space-track-ap2-graph-edges-{leo}.csv"
else:
    edges_savepath = '../datasets/space-track-ap2-graph-edges.csv'
edges_df.to_csv(savepath, index=False)